In [1]:
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.preprocessing import image
from keras.preprocessing import image_dataset_from_directory


In [2]:
#Loading data

train_data = image_dataset_from_directory(
    directory="../data/SkinDisease/train",
    labels="inferred",
    label_mode='categorical',
    color_mode='rgb',
    image_size=(244,244),
    batch_size=32

)

test_data = image_dataset_from_directory(
    directory="../data/SkinDisease/test",
    labels="inferred",
    label_mode='categorical',
    color_mode='rgb',
    image_size=(244,244),
    batch_size=32,
    shuffle=False
)

Found 13898 files belonging to 22 classes.
Found 1546 files belonging to 22 classes.


In [3]:
# Normalizing the pixel values
def normalize(image, label):
    image = tf.cast(image/255., tf.float32)

    return image, label

train_data = train_data.map(normalize)
test_data = test_data.map(normalize)


In [4]:
train_data = train_data.cache().prefetch(buffer_size = tf.data.AUTOTUNE)
test_data = test_data.cache().prefetch(buffer_size = tf.data.AUTOTUNE)

In [5]:
# Building Model

model =  Sequential()

model.add(Conv2D(32, kernel_size=(3,3), padding='same', input_shape=(244,244,3)))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(64, kernel_size=(3,3), padding='same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(128, kernel_size=(3,3), padding='same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(22, activation='softmax'))


d:\practice\skin_disease_detection\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 244, 244, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 122, 122, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 122, 122, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 61, 61, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 61, 61, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 22)             │         1,430 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,662 (56.64 MB)

 Trainable params: 14,848,662 (56.64 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

In [8]:
history = model.fit(train_data, epochs=20, validation_data=test_data, batch_size=64)

Epoch 1/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 885s 2s/step - accuracy: 0.1739 - loss: 2.8410 - val_accuracy: 0.2316 - val_loss: 2.5616
Epoch 2/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 738s 2s/step - accuracy: 0.2403 - loss: 2.4966 - val_accuracy: 0.2924 - val_loss: 2.3336
Epoch 3/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 681s 2s/step - accuracy: 0.2755 - loss: 2.3623 - val_accuracy: 0.2982 - val_loss: 2.2611
Epoch 4/20
324/435 ━━━━━━━━━━━━━━━━━━━━ 2:46 1s/step - accuracy: 0.2929 - loss: 2.2774

KeyboardInterrupt: 